<a href="https://colab.research.google.com/github/harsh-not-haarsh/ASSisted-Driving/blob/master/Obstacle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import skimage.io
import time
import re
import matplotlib
import matplotlib.pyplot as plt 
import sys
import cv2
import numpy as np 
import random
import math
import glob
# root directory 
ROOT_DIR= os.path.abspath("../../")
sys.path.append(ROOT_DIR)
from config import Config
import utils
import model as modellib
import visualize
from model import log

# importing mscoco configurations
#!wget http://images.cocodataset.org/zips/train2017.zip
import coco
#saving logs and trained models
COCO_MODEL_DIR = os.path.join(ROOT_DIR,"logs")
VIDEO_DIR = os.path.join(ROOT_DIR, "media")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "home")
#COCO Weights
COCO_WEIGHTS_FILE=os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")
if not os.path.exists(COCO_WEIGHTS_FILE):
	utils.download_trained_weights(COCO_WEIGHTS_FILE)
#VIDEO_SAVE_DIR=os.path.join(ROOT_DIR,"media")
#Directory of the images
IMAGE_DIR=os.path.join(ROOT_DIR,"media")

# We will be running on inference mode here
class _config_(coco.CocoConfig):
	# since we are running on config mode here we will run one picture at a time
	GPU_COUNT=1
	IMAGES_PER_GPU=1
# Call the above function to implement these assigned features
config=_config_()

# inferencing it
model=modellib.MaskRCNN(mode="inference",model_dir=COCO_MODEL_DIR,config=config)
# Loading weights
model.load_weights(COCO_WEIGHTS_FILE,by_name=True)

class_names=['BG', 'god', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
# Taking an image to test upon
#temp=file_names = next(os.walk(IMAGE_DIR))[2]
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
#results=model.detect([image],verbose=1)
#r=results[0]
#visualize.display_instances(image,r['rois'],r['masks'],r['class_ids'],class_names,r['scores'])
#VIDEO_DIR=os.path.join("ROOT_DIR","media")
capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'vid.mp4.mp4')) 
frame_count=0
frames=[]
batch_size=1
while True:
    ret, frame = capture.read()
    # Bail out when the video file ends
    if not ret:
        break        
    # Save each frame of the video to a list
    
    frame_count += 1
    frames.append(frame)
    if len(frames) == batch_size:
        results = model.detect(frames, verbose=0)
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            frame = display_instances(
                frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
            )
            name = '{0}.jpg'.format(frame_count + i - batch_size)
            name = os.path.join(VIDEO_SAVE_DIR, name)
            cv2.imwrite(name, frame)
        # Clear the frames array to start the next batch
        frames =[]
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by name index.
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

video = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'vid.mp4.mp4'));

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver)  < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

video.release();
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid


import os

# Directory of images to run detection on
#ROOT_DIR = os.getcwd()

#images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
#images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "out.mp4")
make_video(outvid, images, fps=30)

NameError: ignored